In [85]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
import librosa
import resampy
import librosa.display
import IPython.display as ipd
import os
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pydub import AudioSegment
import os
warnings.filterwarnings('ignore')


In [86]:
def mfcc_feature_extractor(audio, sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T, axis=0)

    return mfccsScaledFeatures

In [87]:
def contrast_feature_extractor(audio, sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate)
    contrastScaled = np.mean(contrast.T, axis=0)

    return contrastScaled

In [88]:
def tonnetz_feature_extractor(audio, sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T, axis=0)

    return tonnetzScaled

In [89]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [90]:
def features_extractor(file):
    if file.lower().endswith(".m4a"):
        audio = AudioSegment.from_file(file, format="m4a")
        file_name, _ = os.path.splitext(os.path.basename(file))
        output_path = os.path.join(os.path.dirname(file), file_name + ".wav")
        audio.export(output_path, format="wav")
        # os.remove(file)
        file = output_path

    features = []
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfcc = mfcc_feature_extractor(audio, sample_rate)
    contrast = contrast_feature_extractor(audio, sample_rate)
    tonnetz = tonnetz_feature_extractor(audio, sample_rate)
    chroma = chroma_feature_extractor(audio, sample_rate)

    # Concatenate individual feature arrays
    concatenated_features = np.concatenate((mfcc, contrast, tonnetz, chroma), axis=0)
    features.append(concatenated_features)

    return features

In [91]:
def add_features(extractedFeatures, dirPath, label):
    for file in os.listdir(dirPath):
        filePath = os.path.join(dirPath, file)

        if filePath.lower().endswith(".m4a"):
            audio = AudioSegment.from_file(filePath, format="m4a")
            file_name, _ = os.path.splitext(os.path.basename(filePath))
            output_path = os.path.join(os.path.dirname(filePath), file_name + ".wav")
            audio.export(output_path, format="wav")
            # os.remove(filePath)
            filePath = output_path

        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc = mfcc_feature_extractor(audio, sampleRate)
        contrast = contrast_feature_extractor(audio, sampleRate)
        tonnetz = tonnetz_feature_extractor(audio, sampleRate)
        chroma = chroma_feature_extractor(audio, sampleRate)

        extractedFeatures.append([mfcc, contrast, tonnetz, chroma, label])

In [92]:
dict1 = {
"open":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar",
],

"unlock":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar",
],

"grant":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/omar",

],
"other":[
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmeda",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmedk",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hassan",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ibrahim",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/mohannad",
"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar",
]
}


In [93]:
extractedFeatures = []
for i in dict1.keys():
    for x in dict1[i]:
        add_features(extractedFeatures, x, i)

In [94]:
data = pd.DataFrame(extractedFeatures, columns=['mfcc','contrast','tonnetz','chroma','class'])
print(data.shape)
data.to_csv('password.csv', index=False)
data.describe().T

(307, 5)


,count,unique,top,freq
mfcc,307,307,"[-488.23578, 103.44409, 9.910593, 33.63195, 6....",1
contrast,307,307,"[23.98104457049824, 14.883558570433792, 19.139...",1
tonnetz,307,307,"[-0.01461258018698804, 0.0065578272256597275, ...",1
chroma,307,307,"[0.5946693, 0.6137347, 0.62787664, 0.614527, 0...",1
class,307,4,other,83


In [95]:
features = data.iloc[:, 0:-1]
target = data['class']
target.unique()

array(['open', 'unlock', 'grant', 'other'], dtype=object)

In [96]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

In [97]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

In [98]:
class_counts = np.bincount(target)

for class_label, count in enumerate(class_counts):
    print(f"Class {class_label}: {count} instances")

Class 0: 72 instances
Class 1: 78 instances
Class 2: 83 instances
Class 3: 74 instances


In [99]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=105)

### creating RandomForest model

In [100]:
classifier = RandomForestClassifier(n_estimators=100, criterion="entropy")
classifier.fit(xTrain,yTrain)

RandomForestClassifier(criterion='entropy')

### creating SVM model

In [101]:
# classifier = SVC(kernel='linear',decision_function_shape="ovo" ,probability=True) 
# classifier.fit(xTrain,yTrain)

### creating XGBoost model

In [102]:
# classifier = XGBClassifier(objective='multi:softmax', num_class=4, eval_metric='mlogloss')
# classifier.fit(xTrain, yTrain)

In [103]:
preds = classifier.predict(xTest)

In [104]:
print(classification_report(yTest, preds))

              precision    recall  f1-score   support

           0       0.88      0.83      0.86        18
           1       0.83      0.94      0.88        16
           2       0.83      0.95      0.88        20
           3       0.79      0.65      0.71        23

    accuracy                           0.83        77
   macro avg       0.83      0.84      0.83        77
weighted avg       0.83      0.83      0.83        77



In [105]:
def prediction(pred):
    if pred == 0:
        print("grant")
    elif pred == 1:
        print("open")
    elif pred == 2:
        print("other")
    elif pred == 3:
        print("unlock")

In [106]:
testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/dataset/recorded/2.m4a") 
pred = classifier.predict(testFeatures)
prediction(pred)
print(classifier.predict_proba(testFeatures))

open
[[0.18 0.49 0.12 0.21]]


In [107]:
# testFeatures = features_extractor("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar/omar_open_middle_door_1.wav") # open middle door
# pred = classifier.predict(testFeatures)
# prediction(pred)
# print(classifier.predict_proba(testFeatures))

In [108]:
testFeatures = features_extractor("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad/Recording (57).wav") # unlock the gate
pred = classifier.predict(testFeatures)
prediction(pred)
print(classifier.predict_proba(testFeatures))

unlock
[[0.15 0.06 0.01 0.78]]


In [109]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar")[0:10]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar/{file}") 
  pred = classifier.predict(testFeatures)
  prediction(pred) 
  print(classifier.predict_proba(testFeatures))

open
[[0.04 0.86 0.02 0.08]]
open
[[0.04 0.86 0.02 0.08]]
open
[[0.01 0.94 0.03 0.02]]
open
[[0.01 0.94 0.03 0.02]]
open
[[0.   0.92 0.05 0.03]]
open
[[0.   0.92 0.05 0.03]]
open
[[0.   0.95 0.04 0.01]]
open
[[0.   0.95 0.04 0.01]]
open
[[0.01 0.93 0.04 0.02]]
open
[[0.01 0.93 0.04 0.02]]


In [110]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem/{file}") 
  pred = classifier.predict(testFeatures)
  prediction(pred)
  print(classifier.predict_proba(testFeatures))

open
[[0.04 0.71 0.11 0.14]]
open
[[0.   0.8  0.08 0.12]]
open
[[0.   0.75 0.12 0.13]]
open
[[0.01 0.7  0.14 0.15]]
open
[[0.06 0.47 0.32 0.15]]


In [111]:
pickle_out = open("password.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()